In [1]:
# changing core directory
import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path='../config/my_configs', version_base=None)

cfg = compose(config_name='attack_run_config.yaml')
# print(OmegaConf.to_yaml(cfg))

/cephfs/projects/danikin/.local/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'attack_run_config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


# Imports

In [3]:
import os
import warnings

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader

from src.config import get_attack, get_criterion, get_disc_list, get_model
from src.data import MyDataset, load_data, transform_data
from src.estimation.estimators import AttackEstimator

CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Falling back on slow Cauchy and Vandermonde kernel. Install at least one of pykeops or the CUDA extension for better speed and memory efficiency.


# Initializing

In [5]:
print("Dataset", cfg["dataset"])
X_train, y_train, X_test, y_test = load_data(cfg["dataset"])
X_train, X_test, y_train, y_test = transform_data(
    X_train, X_test, y_train, y_test, slice_data=cfg["slice"]
)

test_loader = DataLoader(
    MyDataset(X_test, y_test), batch_size=cfg["batch_size"], shuffle=False
)

device = torch.device(cfg["cuda"] if torch.cuda.is_available() else "cpu")

attack_model_path = os.path.join(
    cfg["model_folder"],
    cfg["attack_model"]["name"],
    f"model_{cfg['model_id_attack']}_{cfg['dataset']}.pt",
)

attack_model = get_model(
    cfg["attack_model"]["name"],
    cfg["attack_model"]["params"],
    path=attack_model_path,
    device=device,
    train_mode=cfg["attack_model"]["attack_train_mode"],
)

criterion = get_criterion(cfg["criterion_name"], cfg["criterion_params"])

if cfg["use_disc_check"]:
    disc_check_list = get_disc_list(
        model_name=cfg["disc_model_check"]["name"],
        model_params=cfg["disc_model_check"]["params"],
        list_disc_params=cfg["list_check_model_params"],
        device=device,
        path=cfg["disc_path"],
        train_mode=False,
    )
else:
    disc_check_list = None

estimator = AttackEstimator(
    disc_check_list,
    cfg["metric_effect"],
    cfg["metric_hid"],
    batch_size=cfg["estimator_batch_size"],
)

Dataset FordA


In [6]:
alphas = [0]
if "alpha" in cfg["attack"]["attack_params"]:
    alphas = cfg["attack"]["attack_params"]["alpha"]

attacks = list()

for alpha in alphas:
    attack_metrics = pd.DataFrame()
    for eps in cfg["attack"]["attack_params"]["eps"]:
        attack_params = dict(cfg["attack"]["attack_params"])
        attack_params["model"] = attack_model
        attack_params["criterion"] = criterion
        attack_params["estimator"] = estimator
        attack_params["alpha"] = alpha
        attack_params["eps"] = eps

        if "list_reg_model_params" in cfg["attack"]:
            attack_params["disc_models"] = get_disc_list(
                model_name=cfg["disc_model_reg"]["name"],
                model_params=cfg["disc_model_reg"]["params"],
                list_disc_params=cfg["attack"]["list_reg_model_params"],
                device=device,
                path=cfg["disc_path"],
                train_mode=cfg["disc_model_reg"]["attack_train_mode"],
            )

        attacks.append(get_attack(cfg["attack"]["name"], attack_params))

logging


#### Attack and Discriminator choice

In [7]:
disc_model = disc_check_list[0]
attack = attacks[0]

#### Generate Adversarial data

In [8]:
X_orig = test_loader.dataset.X.to(device)
X_adv = attack.apply_attack(test_loader).squeeze().to(device)

X_both = torch.cat([X_orig, X_adv], dim=0)

X_both_if_adv = torch.cat(
    [torch.zeros(X_orig.shape[0]), 
     torch.ones(X_adv.shape[0])]
    )

  0%|          | 0/50 [00:00<?, ?it/s]

#### Dimensionality Reduction

In [9]:
from sklearn.manifold import TSNE

In [10]:
tsne = TSNE(n_components=2)

X_2d = tsne.fit_transform(X_both.cpu())

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f744c2d8f70>
Traceback (most recent call last):
  File "/cephfs/soft/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/cephfs/soft/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/cephfs/soft/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/cephfs/soft/anaconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


# Density estimation

In [11]:
def get_KNN(dist_matrix, n_neigbors=5):
    closest_dist, closest_ind = dist_matrix.topk(n_neigbors+1, largest=False) # n_neigbors+1 т.к. берётся само значение (расстояние 0) и n соседей
    
    return closest_dist.to('cpu'), closest_ind.to('cpu')


def distance_weights(neigh_dist, temp=1):
    weights = torch.nn.functional.softmax(-neigh_dist/temp, dim=1) # тепература берётся, чтобы само значение (расстояние 0) не давало слишком большой вес
    
    return weights


def density(values, neigh_ind, weights):
    
    density_vector = (values[neigh_ind].squeeze() * weights).sum(1)
    return density_vector

In [12]:
disc_preds = disc_model(X_both.unsqueeze(-1))
class_preds = attack_model(X_both.unsqueeze(-1))

In [13]:
distances = torch.cdist(X_both, X_both)

neigh_dist, neigh_ind = get_KNN(distances)

In [14]:
neigh_wgts = distance_weights(neigh_dist, 1)
density_vals = density(disc_preds, neigh_ind, neigh_wgts).cpu().detach().numpy()

# Density Plotting

In [15]:
import plotly.express as px

In [46]:
px.scatter(
    X_2d, 
    x=0, 
    y=1,
    color=density_vals,
    color_continuous_scale=px.colors.named_colorscales()[2],
    width=1200, 
    height=600,
    symbol = X_both_if_adv
)

In [59]:
# px.scatter_3d(
#     X_2d, 
#     x = 0, 
#     y = 1,
#     z = 2, 
#     color = density_vals,
#     color_continuous_scale=px.colors.named_colorscales()[10]
# )